In [44]:
# Ciselne algoritmy - DU3 - Josef Sykora
#
# ECM for factorization v1

def ECM_core(N, B):
    x, y, a, b = get_params(N)
    e_B = get_exponent(B, 0)
    result, flag = square_and_multiply(e_B, (x, y), N, a)
    if flag is None:
        return "Fail_1"
    else:
        if 1 < flag < N:
            return flag, N / flag
        else:
            return "Fail_2"

# choose x, y, a randomly, then calculate b and check if the curve is smooth
def get_params(N):
    x = randint(0, N)
    y = randint(0, N)
    a = randint(0, N)
    b = (y^2 - x^3 - a*x) % N
    while not gcd(4*a^3 + 27*b^2, N) != 1:
        x = randint(0, N)
        y = randint(0, N)
        a = randint(0, N)
        b = (y^2 - x^3 - a*x) % N
    return x, y, a, b

# calculate e_B
def get_exponent(B, start):
    result = 1
    P = Primes()
    p = P.unrank(start)  # indexed from 0
    log_p = floor(log(B,p))
    result *= p^(log_p)
    
    while True:
        p = P.next(p)
        if p > B:
            break
        log_p = floor(log(B, p))
        result *= p^(log_p)
    
    return result

def square_and_multiply(e_B, P, N, a):
    bin_coefs = e_B.digits(2)
    if bin_coefs[0] == 1:
        result = P
    else:
        result = "zero"  # represents zero in the group of the elliptic cuve
    for i in range(1, len(bin_coefs)):
        P, flag = add_on_curve(P, P, N, a)
        if flag is not None:  # use flag if an addition is not defined on the curve
            return P, flag
        if bin_coefs[i] == 1:
            result, flag = add_on_curve(result, P, N, a)
            if flag is not None:
                return P, flag
    return P, None

# add points on the elliptic curve according to addition rules
def add_on_curve(P, Q, N, a):
    if P == "zero":
        return Q, None
    elif Q == "zero":
        return "zero", None
    else:
        x_1, y_1 = P
        x_2, y_2 = Q
        if (y_1 + y_2) % N == 0:
            return "zero", None
        if x_1 != x_2:
            g = gcd(x_2 - x_1, N)
            if g != 1:
                return "zero", g
            else:
                s = (R(y_2) - R(y_1)) / (R(x_2) - R(x_1))
                s = ZZ(s)
                x_3 = s^2 - x_1 - x_2 % N
                y_3 = s*(x_1 - x_3) - y_1
                return (x_3, y_3), None
        else:
            g = gcd(2*y_1, N)
            if g != 1:
                return "zero", g
            else:
                s = (3 * (R(x_1))^2 + R(a)) / (2 * R(y_1))
                s = ZZ(s)
                x_3 = s^2 - x_1 - x_2 % N
                y_3 = s*(x_1 - x_3) - y_1
                return (x_3, y_3), None
                
def factor_num(N, B, max_trys):
    for i in range(max_trys):
        res = ECM_core(my_N, my_B)
        if res is not None and type(res) != type("Fail"):
            print("Factorization successful after", i + 1, "try(s).")
            print(N, "=", res[0], "x", res[1])
            return res
    return "Factorization failed."
    

# 4819 (*), 3869, 3827 (*), 3901 (*), 3713, 3589, 3649 (*), 3569
my_N = 3869
my_B = 10
R = IntegerModRing(my_N)

factor_num(my_N, my_B, 10)
print("Calculations ended.")

Factorization successful after 3 try(s).
3869 = 53 x 73
Calculations ended.
